# 2020L-WUM Praca domowa 2

Kod: **Bartłomiej Eljasiak**

## Załadowanie bibliotek

Z tych bibliotek będziemy korzystać w wielu miejscach, jednak w niektórych fragmentach kodu znajdą się dodatkowe importowania, lecz w takich sytuacjach użytek załadowanej biblioteki jest ograniczony do 'chunku', w którym została załadowana.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn

## Wczytanie danych

In [56]:
# local version 
_data=pd.read_csv('allegro-api-transactions.csv')

#online version
#_data = pd.read_csv('https://www.dropbox.com/s/360xhh2d9lnaek3/allegro-api-transactions.csv?dl=1')
cdata=_data.copy()

### Przyjrzenie się danym

In [3]:
cdata.head()

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
0,0,2016-04-03 21:21:08,4753602474,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,59.99,1,997,0,50177,Warszawa,Komputery
1,1,2016-04-03 15:35:26,4773181874,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,4.90,1,9288,0,12428,Warszawa,"Odzież, Obuwie, Dodatki"
2,2,2016-04-03 14:14:31,4781627074,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,109.90,1,895,0,7389,Leszno,Dom i Ogród
3,3,2016-04-03 19:55:44,4783971474,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",1,1,Matfel1,18.50,0,971,0,15006,Wola Krzysztoporska,Książki i Komiksy
4,4,2016-04-03 18:05:54,4787908274,"['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",1,1,PPHU_RICO,19.90,1,950,0,32975,BIAŁYSTOK,"Odzież, Obuwie, Dodatki"


# Obróbka danych

In [4]:
len(cdata.it_location.unique())

10056

In [5]:
cdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420020 entries, 0 to 420019
Data columns (total 14 columns):
lp                        420020 non-null int64
date                      420020 non-null object
item_id                   420020 non-null int64
categories                420020 non-null object
pay_option_on_delivery    420020 non-null int64
pay_option_transfer       420020 non-null int64
seller                    420020 non-null object
price                     420020 non-null float64
it_is_allegro_standard    420020 non-null int64
it_quantity               420020 non-null int64
it_is_brand_zone          420020 non-null int64
it_seller_rating          420020 non-null int64
it_location               420020 non-null object
main_category             420020 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 36.9+ MB


Na pierwszy rzut oka nie mamy żadnych braków w danych, co znacznie ułatwia nam prace.  

# Kodowanie zmiennych kategorycznych

Wiemy, że naszym targetem będzie `price`, chcemy więc w tym akapicie zamienić wszystkie zmienne kategoryczne, z których w dalszej części kodu będziemy korzystać, na zmienne liczbowe. Skuteczna zamiana przy użyciu odpowiednich metod takich jak **target encoding** oraz **on-hot encoding** pozwoli nam przekształcić obecne informacje, tak byśmy mogli je wykorzystać przy operacjach matematycznych. Pominiemy jednak w naszych przekształceniach kolumnę `categories`. 

## Target encoding dla `it_location`

In [6]:
import category_encoders
y=cdata.price
te = category_encoders.target_encoder.TargetEncoder(cdata.it_location, smoothing=100)
encoded = te.fit_transform(cdata.it_location,y)
encoded

,it_location
0,85.423398
1,85.423398
2,61.991428
3,46.319255
4,117.191953
...,...
420015,28.180792
420016,66.785357
420017,44.276007
420018,106.203076


## Różne rodzaje zakodowania kolumny `main_category`

## One-hot Coding

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# integer encode
le = LabelEncoder()
integer_encoded = le.fit_transform(cdata.main_category)
print(integer_encoded)

# binary encode
onehot_encoder = OneHotEncoder(categories='auto',sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[12 18  6 ... 18  5 15]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Zamieniliśmy w ten sposób kolumnę kategoryczną na 26 kolumn o wartościach 0 lub 1. Nie jest to złe rozwiązanie, jednak stanowczo zwiększa rozmiar naszej ramki i wydłża czas uczenia się modelu. Prawdopodobnie może istnieć lepsze rozwiązanie.

## Helmert Coding
Documentation: [scikit-learn](http://contrib.scikit-learn.org/categorical-encoding/helmert.html)

In [35]:
# Pobieramy go z category_encoders

helmert_encoder = category_encoders.HelmertEncoder()
helmert_encoded = helmert_encoder.fit_transform(cdata.main_category)

#showing only first 5 encoded rows
print(helmert_encoded.loc[1:5,:].transpose())

                    1    2    3    4    5
intercept         1.0  1.0  1.0  1.0  1.0
main_category_0   1.0  0.0  0.0  1.0  1.0
main_category_1  -1.0  2.0  0.0 -1.0 -1.0
main_category_2  -1.0 -1.0  3.0 -1.0 -1.0
main_category_3  -1.0 -1.0 -1.0 -1.0 -1.0
main_category_4  -1.0 -1.0 -1.0 -1.0 -1.0
main_category_5  -1.0 -1.0 -1.0 -1.0 -1.0
main_category_6  -1.0 -1.0 -1.0 -1.0 -1.0
main_category_7  -1.0 -1.0 -1.0 -1.0 -1.0
main_category_8  -1.0 -1.0 -1.0 -1.0 -1.0
main_category_9  -1.0 -1.0 -1.0 -1.0 -1.0
main_category_10 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_11 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_12 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_13 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_14 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_15 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_16 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_17 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_18 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_19 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_20 -1.0 -1.0 -1.0 -1.0 -1.0
main_category_21 -1.0 -1.0 -1.0 -1

## Backward Difference Coding
Documentation: [scikit-learn](http://contrib.scikit-learn.org/categorical-encoding/backward_difference.html#backward-difference-coding)

In [8]:
# Pobieramy go z category_encoders

back_diff_encoder = category_encoders.BackwardDifferenceEncoder()
back_diff_encoded = back_diff_encoder.fit_transform(cdata.main_category)

#showing only first 5 encoded rows
print(back_diff_encoded.loc[1:5,:].transpose())

                         1         2         3         4         5
intercept         1.000000  1.000000  1.000000  1.000000  1.000000
main_category_0   0.037037  0.037037  0.037037  0.037037  0.037037
main_category_1  -0.925926  0.074074  0.074074 -0.925926 -0.925926
main_category_2  -0.888889 -0.888889  0.111111 -0.888889 -0.888889
main_category_3  -0.851852 -0.851852 -0.851852 -0.851852 -0.851852
main_category_4  -0.814815 -0.814815 -0.814815 -0.814815 -0.814815
main_category_5  -0.777778 -0.777778 -0.777778 -0.777778 -0.777778
main_category_6  -0.740741 -0.740741 -0.740741 -0.740741 -0.740741
main_category_7  -0.703704 -0.703704 -0.703704 -0.703704 -0.703704
main_category_8  -0.666667 -0.666667 -0.666667 -0.666667 -0.666667
main_category_9  -0.629630 -0.629630 -0.629630 -0.629630 -0.629630
main_category_10 -0.592593 -0.592593 -0.592593 -0.592593 -0.592593
main_category_11 -0.555556 -0.555556 -0.555556 -0.555556 -0.555556
main_category_12 -0.518519 -0.518519 -0.518519 -0.518519 -0.51

# Uzupełnianie braków

### Wybranie danych z ramki
Dalej będziemy pracowac tylko na 3 kolumnach danych, ograniczę więc je dla przejżystości poczynań.

In [3]:
data_selected= cdata.loc[:,['price','it_seller_rating','it_quantity']]
data_selected.head()

,price,it_seller_rating,it_quantity
0,59.99,50177,997
1,4.90,12428,9288
2,109.90,7389,895
3,18.50,15006,971
4,19.90,32975,950


### Usunięcie danych z kolumny

Dane z kolumny będziemy usuwać funkcją `df.column.sample(frac)` gdzie `frac` będzie oznaczać procent danych, które chcemy zatrzymać. Gwarantuje nam to w miarę losowe usunięcie danych, które powinno być wystarczające do dalszych działań.

In [27]:
cdata.price.sample(frac=0.9)


175798     39.00
48188      30.00
287293      7.29
306515    209.00
132284     49.99
           ...  
360010     59.00
168704     10.00
176364     10.00
173033      3.50
222017    255.00
Name: price, Length: 378018, dtype: float64

### Ocena skutecznosci imputacji
Do oceny skuteczności podanych algorytmów imputacji danych musimy przyjąć jakis sposób liczenia ich. Zgodnie z sugestią prowadzącej skorszystam z [RMSE](https://en.wikipedia.org/wiki/Root_mean_square) czyli root mean square error. Nazwa powinna przybliżyć sposób, którm RMSE jest wyznaczane, jednak ciekawskich zachęcam do wejścia w link.

## Imputacja danych
Napiszmy więc funkcje, która pozwoli nam stestować wybrany sposób imputacji.

In [31]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error

def test_imputation(imputer,iterations=10):
    _resoults=[]
    # we always use the same data, so it's taken globally 
    for i in range(iterations):
        test_data = data_selected.copy()
        test_data.it_seller_rating = test_data.it_seller_rating.sample(frac = 0.9)
        data_imputed = pd.DataFrame(imputer.fit_transform(test_data))
        _resoults.append(np.sqrt(mean_squared_error(data_selected,data_imputed)))
    return _resoults

I to niech będzie przykład działania takiej funkcji

In [32]:
imputer = IterativeImputer(max_iter=10,random_state=0)
RMSE_list = test_imputation(imputer,20)


print("Średnie RMSE wynosi", round(np.mean(RMSE_list)))
print('Odchylenie standardowe RMSE wynosi: ', round(np.std(RMSE_list)))  
RMSE_list

Średnie RMSE wynosi 6659.0
Odchylenie standardowe RMSE wynosi:  64.0


[6632.906128962562,
 6616.972611367261,
 6669.293153632687,
 6667.57123478041,
 6648.684670026652,
 6803.520633176432,
 6659.718673347867,
 6636.639030774161,
 6696.972210439248,
 6612.8746185541,
 6698.791682909619,
 6714.591219568827,
 6517.637274219909,
 6619.267354280276,
 6591.840871038003,
 6561.064086622536,
 6716.837848413626,
 6750.547055769709,
 6663.1712065724005,
 6696.884254545501]

Odchylenie standardowe jest dosyć małe, więc metodę imputacji uważam za skuteczną. Chętnym polecam przetestowanie tej funkcji dla innych typów imputacji oraz zmiennej liczbie iteracji. 

### Usuwanie danych z wielu kolumn

Powtórzmy uprzedni przykład dodając drobną modyfikacje. Tym razem będziemy usuwali dane zarówno z `it_seller_rating` jak i `it_quantity`. Napiszmy do tego odpowiednią funkcję i zobaczmy wyniki.

In [34]:
def test_imputation2(imputer,iterations=10):
    _resoults=[]
    # we always use the same data, so it's taken globally 
    for i in range(iterations):
        test_data = data_selected.copy()
        test_data.it_seller_rating = test_data.it_seller_rating.sample(frac = 0.9)
        test_data.it_quantity = test_data.it_quantity.sample(frac = 0.9)
        data_imputed = pd.DataFrame(imputer.fit_transform(test_data))
        _resoults.append(np.sqrt(mean_squared_error(data_selected,data_imputed)))
    return _resoults 

In [35]:
imputer = IterativeImputer(max_iter=10,random_state=0)
RMSE_list = test_imputation2(imputer,20)

    
print("Średnie RMSE wynosi", round(np.mean(RMSE_list)))
print('Odchylenie standardowe RMSE wynosi: ', round(np.std(RMSE_list)))  
RMSE_list

Średnie RMSE wynosi 7953.0
Odchylenie standardowe RMSE wynosi:  60.0


[7972.091893331043,
 7946.31432792675,
 7926.240254756589,
 8011.989561085879,
 7884.314206512695,
 7883.729585245787,
 7846.029166436529,
 7938.27056966838,
 7993.377395875948,
 8009.851510467814,
 7939.537465840667,
 7842.194990448241,
 8055.04150643776,
 7968.257900988455,
 7988.2824617515935,
 7987.15756366397,
 8048.324862336169,
 7959.979889587243,
 7879.14000574198,
 7976.9210476763055]

Tak jak moglibyśmy się spodziewać, średni błąd jest większy w przypadku gdy usuneliśmy więcej danych. Ponownie zachęcam do powtórzenia obliczeń i sprawdzenia wyników. 

### Spojrzenie na imputacje typu `IterativeImputer`

Wykorzystałem pewien szczególny sposób imputacji danych tzn `IterativeImputer`, o którym nie wspomniałem za dużo podczas korzystania z niego. Chciałbym jednka w tym miejscu go bardziej szczegółowo przedstawić oraz zobaczyć jak liczba iteracji wpływa na jakość imputacji.

Nasz imputer będę chiał przetestować dla całego spektrum wartości `max_iter` i dokładnie to zrobię w poniższej pętli.

**uwaga poniższy kod wykonuję się dosyć długo**

In [40]:
upper_iter_limit = 30
lower_iter_limit = 5
imputation_iterations = 10

mean_RMSE ={
    "single": [],
    "multi": [],
}

for imputer_iterations in range(lower_iter_limit,upper_iter_limit,2):
    _resoults_single = []
    _resoults_multi = []
    imputer = IterativeImputer(max_iter=imputer_iterations,random_state=0)

    print("max_iter: ", imputer_iterations, "/",upper_iter_limit)
    # Data missing from single columns
    _resoults_multi.append(test_imputation(imputer,imputation_iterations))

    # Data missing from multiple column
    _resoults_single.append(test_imputation2(imputer,imputation_iterations))
        
    mean_RMSE['single'].append(np.mean(_resoults_single))
    mean_RMSE['multi'].append(np.mean(_resoults_multi))
        

max_iter:  5 / 30
max_iter:  7 / 30
max_iter:  9 / 30
max_iter:  11 / 30
max_iter:  13 / 30
max_iter:  15 / 30
max_iter:  17 / 30
max_iter:  19 / 30
max_iter:  21 / 30
max_iter:  23 / 30
max_iter:  25 / 30
max_iter:  27 / 30
max_iter:  29 / 30


Przyjrzyjmy się wynikom.

In [55]:
mean_RMSE


{'single': [7922.9610772118995,
  7931.847812258746,
  7939.759375764392,
  7936.595274147573,
  7906.379998655105,
  7936.6230668755115,
  7907.32939577691,
  7923.1550466413555,
  7936.269867054558,
  7920.8966312114835,
  7885.421214156695,
  7966.2242132185365,
  7958.103272938873],
 'multi': [6687.573194618922,
  6676.670097937436,
  6675.955835324938,
  6628.1097755950805,
  6672.175466153951,
  6683.623939644985,
  6668.974658695191,
  6732.501598660665,
  6669.534292890236,
  6691.153098826207,
  6665.3763903465415,
  6674.199929706299,
  6697.665837522693]}

### Komentarz
Co ciekawe nie widać dużej różnicy w błędzie imputacji dla różnych współczynników imputacji. Co wiecej nie ma, żadnego typu korelacji, a więc nie opłaca się brać dużego współczynnika iteracji, ponieważ wcale nie daje on lepszych wyników. Pragnę jednak ograniczyć moje wnioski do tego zbioru, ponieważ nie dysponuję w tym momencie wystarczającą liczbą informacji by twierdzić, że jest to zjawisko globalne. Niech ten przykład posłuży jako pretekst do dalszych dyskusji na ten temat.